<a href="https://colab.research.google.com/github/Sydney-Kelly/Search-For-Movie-Plots/blob/main/03-rag.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Getting Started with RAG using Fireworks Fast Inference LLMs

<a href="https://colab.research.google.com/github/fw-ai/cookbook/blob/main/recipes/rag/rag-paper-titles.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

While large language models (LLMs) show powerful capabilities that power advanced use cases, they suffer from issues such as factual inconsistency and hallucination. Retrieval-augmented generation (RAG) is a powerful approach to enrich LLM capabilities and improve their reliability. RAG involves combining LLMs with external knowledge by enriching the prompt context with relevant information that helps accomplish a task.

This tutorial shows how to getting started with RAG by leveraging vector store and open-source LLMs. To showcase the power of RAG, this use case will cover building a RAG system that suggests short and easy to read ML paper titles from original ML paper titles. Paper tiles can be too technical for a general audience so using RAG to generate short titles based on previously created short titles can make research paper titles more accessible and used for science communication such as in the form of newsletters or blogs.

Before getting started, let's first install the libraries we will use:

In [6]:
%%capture
!pip install chromadb tqdm fireworks-ai python-dotenv pandas
!pip install sentence-transformers
!pip install datasets

Let's download the dataset we will use:

In [8]:
# import data
from datasets import load_dataset
ds = load_dataset("Coder-Dragon/wikipedia-movies", split='train[:1000]')

Generating train split: 0 examples [00:00, ? examples/s]

Before continuing, you need to obtain a Fireworks API Key to use the Mistral 7B model.

Checkout this quick guide to obtain your Fireworks API Key: https://readme.fireworks.ai/docs

In [11]:
import fireworks.client
import os
import dotenv
import chromadb
import json
from tqdm.auto import tqdm
import pandas as pd
import random
from google.colab import userdata




# you can set envs using Colab secrets
fireworks.client.api_key = userdata.get('FIREWORKS_API_KEY')

## Getting Started

Let's define a function to get completions from the Fireworks inference platform.

In [12]:
def get_completion(prompt, model=None, max_tokens=50):

    fw_model_dir = "accounts/fireworks/models/"

    if model is None:
        model = fw_model_dir + "llama-v2-7b"
    else:
        model = fw_model_dir + model

    completion = fireworks.client.Completion.create(
        model=model,
        prompt=prompt,
        max_tokens=max_tokens,
        temperature=0
    )

    return completion.choices[0].text

Let's first try the function with a simple prompt:

In [13]:
get_completion("Hello, my name is")

' Katie and I am a 20 year old student at the University of Leeds. I am currently studying a BA in English Literature and Creative Writing. I have been working as a tutor for over 3 years now and I'

Now let's test with Mistral-7B-Instruct:

In [14]:
mistral_llm = "mistral-7b-instruct-4k"

get_completion("Hello, my name is", model=mistral_llm)

' [Your Name]. I am a [Your Profession/Occupation]. I am writing to [Purpose of Writing].\n\nI am writing to [Purpose of Writing] because [Reason for Writing]. I believe that ['

The Mistral 7B Instruct model needs to be instructed using special instruction tokens `[INST] <instruction> [/INST]` to get the right behavior. You can find more instructions on how to prompt Mistral 7B Instruct here: https://docs.mistral.ai/llm/mistral-instruct-v0.1

In [15]:
mistral_llm = "mistral-7b-instruct-4k"

get_completion("Tell me 2 jokes", model=mistral_llm)

".\n1. Why don't scientists trust atoms? Because they make up everything!\n2. Did you hear about the mathematician who’s afraid of negative numbers? He will stop at nothing to avoid them."

In [16]:
mistral_llm = "mistral-7b-instruct-4k"

get_completion("[INST]Tell me 2 jokes[/INST]", model=mistral_llm)

" Sure, here are two jokes for you:\n\n1. Why don't scientists trust atoms? Because they make up everything!\n2. Why did the tomato turn red? Because it saw the salad dressing!"

Now let's try with a more complex prompt that involves instructions:

In [17]:
prompt = """[INST]
Given the following wedding guest data, write a very short 3-sentences thank you letter:

{
  "name": "John Doe",
  "relationship": "Bride's cousin",
  "hometown": "New York, NY",
  "fun_fact": "Climbed Mount Everest in 2020",
  "attending_with": "Sophia Smith",
  "bride_groom_name": "Tom and Mary"
}

Use only the data provided in the JSON object above.

The senders of the letter is the bride and groom, Tom and Mary.
[/INST]"""

get_completion(prompt, model=mistral_llm, max_tokens=150)

" Dear John Doe,\n\nWe, Tom and Mary, would like to extend our heartfelt gratitude for your attendance at our wedding. It was a pleasure to have you there, and we truly appreciate the effort you made to be a part of our special day.\n\nWe were thrilled to learn about your fun fact - climbing Mount Everest is an incredible accomplishment! We hope you had a safe and memorable journey.\n\nThank you again for joining us on this special occasion. We hope to stay in touch and catch up on all the amazing things you've been up to.\n\nWith love,\n\nTom and Mary"

## RAG Use Case: Movie Titles

For the RAG use case, we will be using [a dataset](https://github.com/dair-ai/ML-Papers-of-the-Week/tree/main/research) that contains a list of weekly top trending ML papers.

The user will provide an original paper title. We will then take that input and then use the dataset to generate a context of short and catchy papers titles that will help generate catchy title for the original input title.



### Step 1: Data Setup


In [18]:
# create movies[] as [title, plot]
movies = []
for title, plot in zip(ds['Title'], ds['Plot']):
  movies.append([title, plot])

# examine movies[]
print(len(movies), "\n")
movies[:3]

1000 



[['Kansas Saloon Smashers',
  "A bartender is working at a saloon, serving drinks to customers. After he fills a stereotypically Irish man's bucket with beer, Carrie Nation and her followers burst inside. They assault the Irish man, pulling his hat over his eyes and then dumping the beer over his head. The group then begin wrecking the bar, smashing the fixtures, mirrors, and breaking the cash register. The bartender then sprays seltzer water in Nation's face before a group of policemen appear and order everybody to leave.[1]"],
 ['Love by the Light of the Moon',
  "The moon, painted with a smiling face hangs over a park at night. A young couple walking past a fence learn on a railing and look up. The moon smiles. They embrace, and the moon's smile gets bigger. They then sit down on a bench by a tree. The moon's view is blocked, causing him to frown. In the last scene, the man fans the woman with his hat because the moon has left the sky and is perched over her shoulder to see everythi

We will be using SentenceTransformer for generating embeddings that we will store to a chroma document store.

In [19]:
from chromadb import Documents, EmbeddingFunction, Embeddings
from sentence_transformers import SentenceTransformer
embedding_model = SentenceTransformer('all-MiniLM-L6-v2')

class MyEmbeddingFunction(EmbeddingFunction):
    def __call__(self, input: Documents) -> Embeddings:
        batch_embeddings = embedding_model.encode(input)
        return batch_embeddings.tolist()

embed_fn = MyEmbeddingFunction()

# Initialize the chromadb directory, and client.
client = chromadb.PersistentClient(path="./chromadb")

# create collection
collection = client.get_or_create_collection(
    name=f"wikipedia-movie-plots"
)

modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/10.7k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/612 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/350 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

We will now generate embeddings for batches:

In [21]:
# Generate embeddings, and index titles in batches
batch_size = 50

# loop through batches and generated + store embeddings
for i in tqdm(range(0, len(movies), batch_size)):

    i_end = min(i + batch_size, len(movies))
    batch = movies[i : i + batch_size]

    # Replace title with "No Title" if empty string
    batch_titles = [str(paper[0]) if str(paper[0]) != "" else "No Title" for paper in batch]
    batch_ids = [str(sum(ord(c) + random.randint(1, 10000) for c in paper[0])) for paper in batch]
    batch_metadata = [dict(plot=paper[1]) for paper in batch]

    # generate embeddings
    batch_embeddings = embedding_model.encode(batch_titles)

    # upsert to chromadb
    collection.upsert(
        ids=batch_ids,
        metadatas=batch_metadata,
        documents=batch_titles,
        embeddings=batch_embeddings.tolist(),
    )

  0%|          | 0/20 [00:00<?, ?it/s]

Now we can test the retriever:

In [22]:
collection = client.get_or_create_collection(
    name=f"wikipedia-movie-plots",
    embedding_function=embed_fn
)

retriever_results = collection.query(
    query_texts=["Documentaries showcasing indigenous peoples' survival and daily life in Arctic regions"
],
    n_results=5,
)

print(retriever_results["documents"])

[['The Frozen North', 'From Leadville to Aspen: A Hold-Up in the Rockies', 'The Ghost of Slumber Mountain', 'The Viking', 'The Call of the Wild']]


Now let's put together our final prompt:

## Query 1

In [36]:
# user query
user_query = "Documentaries showcasing indigenous peoples' survival and daily life in Arctic regions"

# query for user query
results = collection.query(
    query_texts=[user_query],
    n_results=5,
)

# concatenate titles into a single string
short_titles = '\n'.join(results['documents'][0])

prompt_template = f'''[INST]

Your main task is to recommend movies based on the user query.

  You will be provided with the user query and a suggested movie title along with a summary of its plot.

  Generate a 5 recommendations based on the user query and the suggested movie.

  USER_QUERY: {user_query}

  SUGGESTED_TITLES:

[/INST]
'''

responses = get_completion(prompt_template, model=mistral_llm, max_tokens=2000)
suggested_titles = ''.join([str(r) for r in responses])

# Print the suggestions.
print("Model Suggestions:")
print(suggested_titles)
print("\n\n\nPrompt Template:")
print(prompt_template)

Model Suggestions:

1. "The Invisible Warmth" - This documentary explores the lives of indigenous people living in the Arctic region and their unique adaptation to the harsh environment.
2. "The Last Icebound" - This film follows the lives of indigenous communities in the Arctic as they struggle to survive in the face of climate change and melting ice.
3. "The Frozen Frontier" - This documentary showcases the daily lives of indigenous people living in the Arctic and their deep connection to the land and its resources.
4. "The Arctic Frontier" - This film explores the challenges faced by indigenous communities in the Arctic as they navigate the rapidly changing environment and the impact of globalization on their traditional way of life.
5. "The Arctic Survival" - This documentary showcases the incredible resilience and resourcefulness of indigenous people living in the Arctic as they adapt to the extreme conditions and thrive in one of the most inhospitable environments on Earth.



Pr

##Query 2

In [37]:
# user query
user_query = "Western romance"

# query for user query
results = collection.query(
    query_texts=[user_query],
    n_results=5,
)

# concatenate titles into a single string
short_titles = '\n'.join(results['documents'][0])

prompt_template = f'''[INST]

Your main task is to recommend movies based on the user query.

  You will be provided with the user query and a suggested movie title along with a summary of its plot.

  Generate a  5 recommendations based on the user query and the suggested movie.

  USER_QUERY: {user_query}

  SUGGESTED_TITLES:

[/INST]
'''

responses = get_completion(prompt_template, model=mistral_llm, max_tokens=2000)
suggested_titles = ''.join([str(r) for r in responses])

# Print the suggestions.
print("Model Suggestions:")
print(suggested_titles)
print("\n\n\nPrompt Template:")
print(prompt_template)

Model Suggestions:

1. "The Good, the Bad and the Ugly" - A classic Western movie directed by Sergio Leone, starring Clint Eastwood, Eli Wallach, and Lee Van Cleef. The story follows three bandits on a mission to rob a bank in the Wild West.
2. "Butch Cassidy and the Sundance Kid" - A Western movie directed by George Roy Hill, starring Paul Newman and Robert Redford. The story follows two outlaws, Butch Cassidy and the Sundance Kid, as they rob banks and trains across the Wild West.
3. "The Wild Bunch" - A Western movie directed by Sam Peckinpah, starring William Holden, Warren Beatty, and Faye Dunaway. The story follows a group of outlaws, known as The Wild Bunch, as they rob a train and make their way to Mexico.
4. "The Magnificent Seven" - A Western movie directed by John Sturges, starring Yul Brynner, Steve McQueen, and James Coburn. The story follows seven gunslingers hired to protect a town from a group of outlaws.
5. "True Grit" - A Western movie directed by Joel and Ethan Coen,

##Query 3

In [38]:
# user query
user_query = "Silent film about a Parisian star moving to Egypt, leaving her husband for a baron, and later reconciling after finding her family in poverty in Cairo."

# query for user query
results = collection.query(
    query_texts=[user_query],
    n_results=5,
)

# concatenate titles into a single string
short_titles = '\n'.join(results['documents'][0])

prompt_template = f'''[INST]

Your main task is to recommend movies based on the user query.

  You will be provided with the user query and a suggested movie title along with a summary of its plot.

  Generate a 5 recommendations based on the user query and the suggested movie.

  USER_QUERY: {user_query}

  SUGGESTED_TITLES:

[/INST]
'''

responses = get_completion(prompt_template, model=mistral_llm, max_tokens=2000)
suggested_titles = ''.join([str(r) for r in responses])

# Print the suggestions.
print("Model Suggestions:")
print(suggested_titles)
print("\n\n\nPrompt Template:")
print(prompt_template)

Model Suggestions:

1. "The Lady of the Camelias" (1921) - A romantic drama about a Parisian woman who falls in love with a wealthy Egyptian baron and must navigate the social norms of both societies.
2. "The Egyptian" (1922) - A thrilling adventure about a wealthy American woman who becomes embroiled in a dangerous political upheaval in Egypt and must use her wits to survive.
3. "The Scarlet Empress" (1934) - A historical drama about the rise of Catherine the Great and her tumultuous relationship with her husband, Peter the Great.
4. "The Phantom of Liberty" (1926) - A surrealistic horror film about a man who becomes trapped in a dreamlike state and must navigate a world of bizarre and terrifying creatures.
5. "The Passion of Joan of Arc" (1928) - A powerful and emotional drama about the trial and execution of Joan of Arc, the French heroine who fought for France during the Hundred Years' War.



Prompt Template:
[INST]

Your main task is to recommend movies based on the user query.



##Query 4

In [39]:
# user query
user_query = "Lost film, Cleopatra charms Caesar, plots world rule, treasures from mummy, revels with Antony, tragic end with serpent in Alexandria."
# query for user query
results = collection.query(
    query_texts=[user_query],
    n_results=5,
)

# concatenate titles into a single string
short_titles = '\n'.join(results['documents'][0])

prompt_template = f'''[INST]

Your main task is to recommend movies based on the user query.

  You will be provided with the user query and a suggested movie title along with a summary of its plot.

  Generate a 5 recommendations based on the user query and the suggested movie.

  USER_QUERY: {user_query}

  SUGGESTED_TITLES:

[/INST]
'''

responses = get_completion(prompt_template, model=mistral_llm, max_tokens=2000)
suggested_titles = ''.join([str(r) for r in responses])

# Print the suggestions.
print("Model Suggestions:")
print(suggested_titles)
print("\n\n\nPrompt Template:")
print(prompt_template)

Model Suggestions:

1. "Cleopatra" (1963) - Directed by Joseph L. Mankiewicz, this epic drama film follows the life of the last pharaoh of Ancient Egypt, Cleopatra, and her romantic relationships with Julius Caesar and Mark Antony.
2. "The Mummy" (1999) - Directed by Stephen Sommers, this action-adventure film follows a group of archaeologists who uncover the tomb of an ancient Egyptian mummy and must fight off his curse to protect themselves and the world.
3. "The Serpent and the Rainbow" (1985) - Directed by John Landis, this thriller film follows a detective who investigates the mysterious death of a voodoo priest in Haiti and uncovers a plot to bring the ancient god of death back to life.
4. "The Cleopatra Project" (2006) - Directed by James A. Robinson, this science fiction film follows a group of scientists who travel back in time to ancient Egypt to prevent Cleopatra from assassinating Julius Caesar and changing the course of history.
5. "The Mummy: Tomb of the Dragon Emperor" (

##Query 5

In [40]:
# user query
user_query = "Comedy film, office disguises, boss's daughter, elopement."
# query for user query
results = collection.query(
    query_texts=[user_query],
    n_results=5,
)

# concatenate titles into a single string
short_titles = '\n'.join(results['documents'][0])

prompt_template = f'''[INST]

Your main task is to recommend movies based on the user query.

  You will be provided with the user query and a suggested movie title along with a summary of its plot.

  Generate a 5 recommendations based on the user query and the suggested movie.

  USER_QUERY: {user_query}

  SUGGESTED_TITLES:

[/INST]
'''

responses = get_completion(prompt_template, model=mistral_llm, max_tokens=2000)
suggested_titles = ''.join([str(r) for r in responses])

# Print the suggestions.
print("Model Suggestions:")
print(suggested_titles)
print("\n\n\nPrompt Template:")
print(prompt_template)

Model Suggestions:

1. "The Proposal" - A romantic comedy film about a man who disguises himself as his boss to convince his boss's daughter to marry him.
2. "Office Space" - A satirical comedy film about a group of office workers who decide to take matters into their own hands and stage an office elopement.
3. "The Princess Diaries" - A comedy film about a high school student who disguises herself as the princess of a small European country to attend a royal ball.
4. "My Best Friend's Wedding" - A romantic comedy film about a woman who disguises herself as her best friend's fiancé to prevent her from marrying someone else.
5. "The Wedding Singer" - A romantic comedy film about a wedding singer who disguises himself as a wealthy businessman to win the heart of a woman he has been singing to at weddings.



Prompt Template:
[INST]

Your main task is to recommend movies based on the user query.

  You will be provided with the user query and a suggested movie title along with a summary o

##Query 6

In [42]:
# user query
user_query = "Denis Gage Deane-Tanner."
# query for user query
results = collection.query(
    query_texts=[user_query],
    n_results=5,
)

# concatenate titles into a single string
short_titles = '\n'.join(results['documents'][0])

prompt_template = f'''[INST]

Your main task is to recommend movies based on the user query.

  You will be provided with the user query and a suggested movie title along with a summary of its plot.

  Generate a 5 recommendations based on the user query and the suggested movie.

  USER_QUERY: {user_query}

  SUGGESTED_TITLES:

[/INST]
'''

responses = get_completion(prompt_template, model=mistral_llm, max_tokens=2000)
suggested_titles = ''.join([str(r) for r in responses])

# Print the suggestions.
print("Model Suggestions:")
print(suggested_titles)
print("\n\n\nPrompt Template:")
print(prompt_template)

Model Suggestions:

1. "The Fighter" (2010) - This biographical drama film tells the story of boxer Micky Ward, who overcomes numerous obstacles to become a world champion.
2. "The Wrestler" (2008) - This drama film follows the life of a retired professional wrestler, Mickey Rourke, as he struggles to find meaning and purpose in his life.
3. "The Rocky Balboa" (2006) - This sports drama film follows the story of a small-time boxer, Rocky Balboa, who gets a shot at the world heavyweight championship.
4. "The Raging Bull" (1980) - This biographical drama film tells the story of boxer Jake LaMotta, who becomes increasingly violent and self-destructive as he rises to fame.
5. "The Hurricane" (1999) - This biographical drama film follows the story of boxer Rubin "Hurricane" Carter, who is wrongfully convicted of murder and spends 19 years in prison before being released and regaining his title as a world champion.



Prompt Template:
[INST]

Your main task is to recommend movies based on th

As you can see, the short titles generated by the LLM are somewhat okay. This use case still needs a lot more work and could potentially benefit from finetuning as well. For the purpose of this tutorial, we have provided a simple application of RAG using open-source models from Firework's blazing-fast models.

Try out other open-source models here: https://app.fireworks.ai/models

Read more about the Fireworks APIs here: https://readme.fireworks.ai/reference/createchatcompletion
